In [2]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

### Read the data for January. How many columns are there?

In [3]:
df = pd.read_parquet('/content/drive/MyDrive/Estudo/MLOps Zoomcamp/Homework_01/yellow_tripdata_2023-01.parquet')
print(len(df.columns))

19


### What's the standard deviation of the trips duration in January?

In [4]:
df.head(3)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.0,0.5,0.0,0.0,1.0,14.3,2.5,0.0
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.0,0.5,4.0,0.0,1.0,16.9,2.5,0.0
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.0,0.5,15.0,0.0,1.0,34.9,2.5,0.0


In [5]:
df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime'] # Nova coluna com a duração da viagem
df['duration'] = df.duration.apply(lambda td: td.total_seconds() / 60) # Converter a coluna para segundos

In [6]:
df['duration'].describe().round(2)

count    3066766.00
mean          15.67
std           42.59
min          -29.20
25%            7.12
50%           11.52
75%           18.30
max        10029.18
Name: duration, dtype: float64

### What fraction of the records left after you dropped the outliers?

In [7]:
len_before = len(df)
print(len_before)

3066766


In [8]:
df2 = df[(df.duration >= 1) & (df.duration <= 60)] # Filtro para deixar apenas viagens entre 1 minuto e 1 hora.


In [9]:
len_after = len(df2)
print(len_after)

3009173


In [12]:
tot_used = round((len_after * 100) / len_before)
print(f"Total used = {tot_used}%")

Total used = 98%


### What's the dimensionality of this matrix (number of columns)?

In [14]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df2[categorical] = df2[categorical].astype(str)

df2.head(3)

<ipython-input-14-f2747c052338>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[categorical] = df2[categorical].astype(str)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.0,0.5,0.0,0.0,1.0,14.3,2.5,0.0,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.0,0.5,4.0,0.0,1.0,16.9,2.5,0.0,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.0,0.5,15.0,0.0,1.0,34.9,2.5,0.0,12.750000


In [15]:
train_dicts = df2[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df2[target].values



In [19]:
X_train.shape

(3009173, 516)

### What's the RMSE on train?

In [20]:
lr = LinearRegression()
lr.fit(X_train,y_train)

y_pred = lr.predict(X_train)

mse = mean_squared_error(y_train, y_pred, squared=False)

print(mse)

7.656752063242953


In [21]:
df_train = df2.copy()

In [22]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']

In [ ]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [ ]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared = False)

In [ ]:
with open('/content/drive/MyDrive/Estudo/MLOps Zoomcamp/models/lin_reg.bin', 'wb') as f_out:
  pickle.dump((dv, lr), f_out)

In [ ]:
lr = Lasso(0.01)# Ridge
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)